# Assignment 4
by Himani Anil Deshpande

In [1]:
pip install py4j

     |████████████████████████████████| 198 kB 11.6 MB/s 


In [2]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 38 kB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ee26c12c874e6ada4c92f46c4d28cd1c96d135c1fc8cc33cc9f302587b591994
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Connecting to Spark Cluster

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.sql.functions import col, asc,desc
import json
import csv
from io import StringIO
from pyspark.sql import HiveContext
sc = SparkContext()

Load fake_data.csv

In [4]:

# datafile = "file:///Users/himanideshpande/miniforge3/envs/ENV_SPARK/SparkInstallation/spark-3.2.1-bin-hadoop2.7/IPYNB/Fake_data.csv"
datafile = "/content/Fake_data.csv"
# input_json = sc.textFile(datafile)
def loadRecord(line):
    """Parse a CSV line"""
    input = StringIO(line)
    reader = csv.DictReader(input, fieldnames=["index","Birth_Country","Email","First_Name","Income","Job","Last_name","Loan_Approved","SSN"])
    return next(reader)

input_data = sc.textFile(datafile).map(loadRecord)


In [5]:
rdd = sc.parallelize(input_data.collect())

In [6]:
# rdd.collect()

spark = SparkSession(sc)

In [7]:
df =  rdd.toDF()

In [8]:
df.printSchema()

root
 |-- Birth_Country: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Job: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Loan_Approved: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- index: string (nullable = true)



In [9]:
df = df.withColumn('Income', col('Income').cast(IntegerType()))

In [10]:
df.printSchema()

root
 |-- Birth_Country: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Loan_Approved: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- index: string (nullable = true)



In [11]:
type(rdd)

pyspark.rdd.RDD

Question 1. 
Find birth country which has highest amount of people 

In [12]:
highest_df = df.groupBy("Birth_Country").count()

In [13]:
highest_df.orderBy(col("count").desc()).show(1)

+-------------+-----+
|Birth_Country|count|
+-------------+-----+
|        Korea|   91|
+-------------+-----+
only showing top 1 row



Question 2
Find average income of people who are born in united states of america 

In [14]:
df.filter(df['Birth_Country'] =='United States of America').groupBy('Birth_Country').avg('Income').show()

+--------------------+------------------+
|       Birth_Country|       avg(Income)|
+--------------------+------------------+
|United States of ...|208759.82352941178|
+--------------------+------------------+



Question 3 How many people has income over 100,000 but their loan is not approved. 

In [15]:
df_count_income_high = df.filter((df.Income >100000) & (df.Loan_Approved == False))

In [16]:
df_count_income_high.count()

4009

Question 4 Find top 10 people with highest income in United States of america. (Print their names, income and jobs)

In [23]:
top_paid = df.filter(df['Birth_Country'] =='United States of America').orderBy(col("Income").desc())

In [24]:
top_paid.select("First_Name", "Last_name","Job", "Income").show(10)

+----------+---------+--------------------+------+
|First_Name|Last_name|                 Job|Income|
+----------+---------+--------------------+------+
|    Alyssa|   Miller|Amenity horticult...|482588|
|    Hunter|    Walls|Psychologist, pri...|468946|
|      Rose|Henderson|Adult guidance wo...|426115|
|  Danielle|  Leonard|Furniture conserv...|389810|
|     Terry|    Klein|       Meteorologist|380410|
|     Cindy|   Newton|Research scientis...|370322|
|     Scott| Mitchell|       Art therapist|368913|
|   Christy| Sandoval|      Engineer, land|355150|
|     Kelly| Reynolds|           Press sub|341448|
|  Kristina|    Smith|           Herbalist|338804|
+----------+---------+--------------------+------+
only showing top 10 rows



Question 5 How many number of distinct jobs are there?

In [25]:
df.select("Job").distinct().count()

640

Question 6 How many writers earn less than 100,000? 

In [26]:
df.filter((df.Income <100000) & (df.Job == 'Writer')).count()

5

References:

https://www.datasciencemadesimple.com/distinct-value-of-a-column-in-pyspark/#:~:text=Distinct%20value%20of%20the%20column%20in%20pyspark%20is%20obtained%20by,value%20of%20those%20columns%20combined.